In [1]:
import os

In [2]:
%pwd

'/home/shrey/Desktop/Kidney-Disease-Classifcation/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/shrey/Desktop/Kidney-Disease-Classifcation'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Shrey-patel-07/Kidney-Disease-Classifcation.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Shrey-patel-07"
os.environ["MLFLOW_TRACKING_PASSWORD"]="6a425a20bb5b645c5efa1ca49ab28097a92a76ee"

In [6]:
import tensorflow as tf

2024-01-04 12:24:18.257148: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-04 12:24:18.258826: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-04 12:24:18.285564: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-04 12:24:18.285595: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-04 12:24:18.286297: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [7]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

2024-01-04 12:24:20.808055: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
os.chdir('src/')

In [10]:
from kidney_classification.constants import *
from kidney_classification.utils.common import read_yaml, create_directories, save_json

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        param_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(param_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone",
            mlflow_uri="https://dagshub.com/Shrey-patel-07/Kidney-Disease-Classifcation.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [12]:
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [13]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.valid_generator = None  # Initialize to None

    def _valid_generator(self):
        img_height, img_width = self.config.params_image_size[:-1]

        self.valid_generator = tf.keras.utils.image_dataset_from_directory(
            self.config.training_data,
            image_size=(img_height, img_width),
            validation_split=0.30,
            subset='validation',
            seed=123
        )

        self.valid_generator = self.valid_generator.map(lambda x, y: (x / 255, y))
        AUTOTUNE = tf.data.AUTOTUNE
        self.valid_generator = self.valid_generator.cache().prefetch(buffer_size=AUTOTUNE)


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [14]:
os.chdir('../')

In [15]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
    
except Exception as e:
   raise e

[2024-01-04 12:24:51,908: INFO: common yaml file: config/config.yaml loaded successfully]
[2024-01-04 12:24:51,910: INFO: common yaml file: params.yaml loaded successfully]
Found 12446 files belonging to 1 classes.
Using 3733 files for validation.
117/117 [==============================] - 147s 1s/step - loss: 3.2269e-06 - accuracy: 1.0000
[2024-01-04 12:27:19,911: INFO: common json file saved at: scores.json]
[2024-01-04 12:27:19,912: INFO: common json file saved at: scores.json]


2024/01/04 12:27:22 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /tmp/tmpdrlg6a4p/model/data/model/assets
[2024-01-04 12:27:23,234: INFO: builder_impl Assets written to: /tmp/tmpdrlg6a4p/model/data/model/assets]


/home/shrey/Desktop/Kidney-Disease-Classifcation/env/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/01/04 12:27:47 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 3
Created version '3' of model 'VGG16Model'.
